In [183]:
# Importo librerías

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import Counter

In [184]:
# Obtengo credenciales

client_credentials_manager = SpotifyClientCredentials("bcd05cdd915c49429590886e25342140", 
                                                      "0eae1090f5c347769d17b38d4892e303")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [185]:
# De la aplicación voy a recibir una lista de nombres de usuarios
# Tomo la siguiente lista como ejemplo:

users = ["sedepa", "zltm3cqxt0u0s9k4e8oy564dk"]

In [186]:
# Para el paso 3 del layout de la app, voy a tener que devolver a cada usuario una lista con 
# el nombre de sus playlists, para que seleccione cuáles va a querer subir para crear la lista 
# conjunta.

# Por tanto, de la siguiente función voy a obtener un diccionario para devolver a la app, con
# el siguiente formato:

diccionario1 = {"user1": 
                   ["playlist1_name", "playlist2_name"],
                "user2": 
                   ["playlist1_name", "playlist2_name"]}


# Además, aunque al usuario en la app no se lo voy a mostrar, también voy a aprovechar para 
# obtener aquí el playlist_ID correspondiente a cada playlist, guardándolos en otro diccionario 
# con el siguiente formato:


diccionario2 = {"user1": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"},
                "user2": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"}}

In [187]:
# Defino la función que obtenga ambos diccionarios a partir de una lista de usuarios.

def get_user_playlists(users):
    
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    
    for user in users:
        playlists_names = []
        users_playlists_names_IDs[user] = {}
        playlists = sp.user_playlists(user)
        
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
            
        users_playlists_names[user] = playlists_names
        
    return users_playlists_names, users_playlists_names_IDs


In [188]:
# Guardo cada diccionario en una variable

users_playlists_names = get_user_playlists(users)[0]
users_playlists_names_IDs = get_user_playlists(users)[1]

In [189]:
# El diccionario users_playlists_names va a ser devuelto a la app para que cada usuario
# seleccione de entre sus playlists cuáles quiere subir.

# Además, cada usuario va a poder filtrar las canciones de sus playlists seleccionadas 
# por los géneros que escoja.

# También van a tener que indicar cuál de ellos será el "host" (el usuario en cuyo perfil 
# se creará la lista conjunta), y el nombre de la lista conjunta. 

In [190]:
# Después de que los usuarios hayan seleccionado las playlists que quieren subir, el género
# de las canciones, el host y el nombre de la nueva lista, tengo que recibir 
# un diccionario con el siguiente formato:

mygroup = {"users": 
                   {"user1": {"playlists": [], "genres": []}, 
                    "user2": {"playlists": [], "genres": []}}, 
           "host": "user1",
           "new_playlist_name": "name"}

# Es posible que las listas de valores de "genres" y "playlists" estén vacías,
# lo cual significa que el usuario no quiere filtrar por género, y quiere seleccionar 
# todas las canciones de las playlists que sube. 

In [191]:
# Tomo los siguientes grupos como ejemplos:

mygroup1 = {"users": 
                   {"sedepa": {"playlists": [],
                               "genres": []}, 
                    "zltm3cqxt0u0s9k4e8oy564dk": {"playlists": ["FPP"],
                                                  "genres": []}}, 
           "host": "zltm3cqxt0u0s9k4e8oy564dk",
           "new_playlist_name": "ÉXITO"}

mygroup2 = {"users": 
                   {"sedepa": {"playlists": [],
                               "genres": []}, 
                    "zltm3cqxt0u0s9k4e8oy564dk": {"playlists": ["Final Project"],
                                                  "genres": []}}, 
           "host": "zltm3cqxt0u0s9k4e8oy564dk",
           "new_playlist_name": "ÉXITO"}

In [192]:
b = []

for user in mygroup1["users"].keys():
    if mygroup1["users"][user]["playlists"] == []:
        for playlist_ID in users_playlists_names_IDs[user].values():
            b.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
    else:
        for playlist in mygroup1["users"][user]["playlists"]:
                b.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))

In [193]:
len(b)

35

In [194]:
# Defino una función que obtenga una lista con todas las canciones que hay en todas las 
# playlist que los usuarios me han pasado, filtrando por género en caso de que así lo
# hayan especificado.
# Para ello, primero tengo que obtener el ID correspondiente a cada nombre de playlist
# que me hayan pasado.

def get_tracks_info(group):
    tracks_info = []
    for user in group["users"].keys():
        
        if group["users"][user]["playlists"] == []:
            for playlist_ID in users_playlists_names_IDs[user].values():
                tracks_info.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
        
        else:
            for playlist in group["users"][user]["playlists"]:
                tracks_info.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))
        
    return tracks_info

In [195]:
tracks_info = get_tracks_info(mygroup1)

In [196]:
len(tracks_info)

35

In [197]:
final_track_names = []
final_track_IDs = []

for playlist in tracks_info:
    for track in playlist["items"]:
        final_track_names.append(track["track"]["name"])
        final_track_IDs.append(track["track"]["id"])    

In [198]:
final_track_names = [item for items, c in Counter(final_track_names).most_common() for item in [items] * c]

In [199]:
final_track_names

['I Feel It Coming',
 'I Feel It Coming',
 'I Feel It Coming',
 'I Feel It Coming',
 'I Feel It Coming',
 'No Buses',
 'No Buses',
 'No Buses',
 'No Buses',
 'Figures',
 'Figures',
 'Figures',
 'Figures',
 'The Night We Met',
 'The Night We Met',
 'The Night We Met',
 'The Night We Met',
 'Tomorrow Morning',
 'Tomorrow Morning',
 'Tomorrow Morning',
 'Tomorrow Morning',
 'Crown of Love',
 'Crown of Love',
 'Crown of Love',
 'Same Drugs',
 'Same Drugs',
 'Same Drugs',
 "Life's A Beach",
 "Life's A Beach",
 "Life's A Beach",
 'Haunted By You',
 'Haunted By You',
 'Haunted By You',
 'Up From Below',
 'Up From Below',
 'Up From Below',
 'Passionfruit',
 'Passionfruit',
 'Passionfruit',
 'My Love 4 U',
 'My Love 4 U',
 'My Love 4 U',
 'Show Me Love (feat. Chance The Rapper, Moses Sumney and Robin Hannibal) - Skrillex Remix',
 'Show Me Love (feat. Chance The Rapper, Moses Sumney and Robin Hannibal) - Skrillex Remix',
 'Show Me Love (feat. Chance The Rapper, Moses Sumney and Robin Hannibal) -

In [11]:
# Estoy intentando encontrar la forma de filtrar por géneros:

artist = sp.artist("5Hsv8dUHHOdnn72q4XIVz7")
artist

# He descubierto que es a través del artista.

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Hsv8dUHHOdnn72q4XIVz7'},
 'followers': {'href': None, 'total': 317637},
 'genres': ['latin arena pop', 'spanish pop', 'spanish rock'],
 'href': 'https://api.spotify.com/v1/artists/5Hsv8dUHHOdnn72q4XIVz7',
 'id': '5Hsv8dUHHOdnn72q4XIVz7',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/b1607026e22f716f1455c11f40ac0d3fb037e320',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/c47d63e9cacda8e6e9477ffc9b5e854dc94ce251',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/8dc8bdbfbe741ea720e10da4a70d251b7178f1c8',
   'width': 160}],
 'name': 'Leiva',
 'popularity': 72,
 'type': 'artist',
 'uri': 'spotify:artist:5Hsv8dUHHOdnn72q4XIVz7'}

In [13]:
if "spanish pop" in artist["genres"]:
    print("yes")
    

yes


In [ ]:
genres = ['deep house', 'minimal tech house', 'tropical house', 'spanish pop', 'spanish rock']

In [60]:
# Defino la función para obtener los temas de una playlist.

tracks = sp.user_playlist_tracks("zltm3cqxt0u0s9k4e8oy564dk", 
                                 playlist_id= "6sHkEyFhxTQIDxCQOZYAns", 
                                 fields=None, limit=500, offset=0, market=None)